In [ ]:
!pip install python-terrier lightgbm==2.2.3

In [2]:
import pyterrier as pt
if not pt.started():
  pt.init()


import lightgbm
assert lightgbm.__version__ == '2.2.3'

terrier-assemblies 5.6  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.7.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


In [3]:
# patch location of topics and qrels
def _filter_on_qid_type(self, component, variant):
  import pandas as pd
  if component == "topics":
    data = self.get_topics("all")
  elif component == "qrels":
    data = self.get_qrels("all")
  qid2type = pd.read_csv("http://mirror.ir-datasets.com/79737768b3be1aa07b14691aa54802c5", names=["qid", "type"], sep=" ")
  qid2type["qid"] = qid2type.apply(lambda row: row["qid"].split("-")[1], axis=1)
  rtr = data.merge(qid2type[qid2type["type"] == variant], on=["qid"])
  if len(rtr) == 0:
    raise ValueError("No such topic type '%s'" % variant)
  rtr.drop(columns=['type'], inplace=True)
  return (rtr, "direct")

dataset = pt.get_dataset("trec-wt-2004")
for t in ["np", "td", "hp"]:
  dataset.locations["qrels"][t] = _filter_on_qid_type
  dataset.locations["topics"][t] = _filter_on_qid_type
dataset.locations["qrels"]["all"] = ('04.qrels.web.mixed.txt', "http://www.dcs.gla.ac.uk/~craigm/04.qrels.web.mixed.txt")
dataset.locations["topics_prefixed"]["all"] = ('Web2004.query.stream.trecformat.txt', "http://www.dcs.gla.ac.uk/~craigm/Web2004.query.stream.trecformat.txt", "trec")

In [4]:
UNAME="2549408b"
PWORD="335cf17f"

from pyterrier.datasets import STANDARD_TERRIER_INDEX_FILES, RemoteDataset

#  "50pct" and "trec-wt-2004" datasets
Fiftypct = pt.get_dataset("50pct",  user=UNAME, password=PWORD)
dotgov_topicsqrels = pt.get_dataset("trec-wt-2004")

In [5]:
def removeEx2Index():
  import os
  indexdir = os.path.join(Fiftypct.corpus_home, "index")
  if os.path.exists(os.path.join(indexdir, "data.properties")) and not os.path.exists(os.path.join(indexdir, "data-pagerank.oos")):
     
    print("WARNING: I have detected and removed an Ex2 index - if you are still working on Ex2, results will be identical, but " +
          "querying time will be a bit longer")
    print("To restore the original Ex2 index, you can delete %s and rerun the Ex2 notebook" % indexdir)
    import shutil
    shutil.rmtree(indexdir)

removeEx2Index()

indexref = Fiftypct.get_index(variant="ex2")
index = pt.IndexFactory.of(indexref)


data.meta-0.fsomapfile:   0%|          | 0.00/50.1M [00:00<?, ?iB/s]

data-pagerank.oos:   0%|          | 0.00/9.52M [00:00<?, ?iB/s]

data.direct.bf:   0%|          | 0.00/1.29G [00:00<?, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/19.3M [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/1.30G [00:00<?, ?iB/s]

...

data.lexicon.fsomapid:   0%|          | 0.00/7.80M [00:00<?, ?iB/s]

data.meta.idx:   0%|          | 0.00/6.16M [00:00<?, ?iB/s]

data.meta.zdata:   0%|          | 0.00/861M [00:00<?, ?iB/s]

data.properties:   0%|          | 0.00/4.36k [00:00<?, ?iB/s]

17:31:31.498 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 860.9 MiB of memory would be required.


In [7]:
print(index.getCollectionStatistics())
print("In the meta index: " + str(index.getMetaIndex().getKeys()))

Number of documents: 807775
Number of terms: 2043788
Number of postings: 177737957
Number of fields: 2
Number of tokens: 572916194
Field names: [TITLE, ELSE]
Positions:   true

In the meta index: ['docno', 'url', 'title', 'body']


In [8]:
tr_topics = Fiftypct.get_topics("training")
va_topics = Fiftypct.get_topics("validation")

tr_qrels = Fiftypct.get_qrels("training")
va_qrels = Fiftypct.get_qrels("validation")

test_topics = dotgov_topicsqrels.get_topics("hp")
test_qrels = dotgov_topicsqrels.get_qrels("hp")

training.topics:   0%|          | 0.00/7.75k [00:00<?, ?iB/s]

validation.topics:   0%|          | 0.00/4.39k [00:00<?, ?iB/s]

training.qrels:   0%|          | 0.00/2.88k [00:00<?, ?iB/s]

validation.qrels:   0%|          | 0.00/1.48k [00:00<?, ?iB/s]

Web2004.query.stream.trecformat.txt:   0%|          | 0.00/15.3k [00:00<?, ?iB/s]

04.qrels.web.mixed.txt:   0%|          | 0.00/1.90M [00:00<?, ?iB/s]

In [9]:
firstpassUB = pt.BatchRetrieve(index, wmodel="PL2", metadata=["docno", "url", "body"], verbose=True) #Baseline setup


In [14]:
firstpassUB.search("chemical reactions")

BR(PL2):   0%|          | 0/1 [00:00<?, ?q/s]

,qid,docid,docno,url,body,rank,score,query
0,1,513586,G18-38-1767991,http://www.boulder.nist.gov/div838/tar/file03....,NIST - Physical and Chemical Properties Divi...,0,12.755546,chemical reactions
1,1,38544,G01-14-2537005,http://www.labtrain.noaa.gov/shemtfa/chemhaz/n...,. ...,1,11.906524,chemical reactions
2,1,707122,G26-06-3754605,http://www.aps.anl.gov/xfd/tech/safetyenvelope...,APS Experiment Safety Envelope 6: Chemicals ...,2,11.877550,chemical reactions
3,1,382754,G13-59-3981168,http://response.restoration.noaa.gov/chemaids/...,"""); } else { document.write(...",3,11.858475,chemical reactions
4,1,70292,G02-16-2617043,http://www.symp14.nist.gov/PDF/COR04MAY.PDF,A Database of Chemical Reactions Designed to A...,4,11.731490,chemical reactions
...,...,...,...,...,...,...,...,...
995,1,246965,G08-68-4141101,http://en-env.llnl.gov/asd/pinatub.html,The Chemical and Radiative Effects of the Moun...,995,6.290707,chemical reactions
996,1,611136,G22-04-3955177,http://eospso.gsfc.nasa.gov/ftp_docs/Ch7.pdf,Chapter 7 ...,996,6.289822,chemical reactions
997,1,594957,G21-38-0191596,http://www.oit.doe.gov/news/oittimes/wn02/wn02...,search ...,997,6.287830,chemical reactions
998,1,280944,G09-85-3411646,http://www.ig.doe.gov/pdf/chemfina.pdf,INS-O-00-01 I N S P E C T I O N ...,998,6.287759,chemical reactions


In [ ]:
pagerankfile = indexref.toString().replace(".properties", "-pagerank.oos")
features = [
    "SAMPLE", #ie PL2
    "WMODEL:SingleFieldModel(BM25,0)", #BM25 title
    "QI:StaticFeature(OIS,%s)" % pagerankfile,
]

stdfeatures = pt.FeaturesBatchRetrieve(index, features, verbose=True)
stage12 = firstpassUB >> stdfeatures #list of features needed to deploy baseline learning to rank 

In [ ]:
stage12.search("chemical reactions").head(2) #new feature column

In [ ]:
stage12.search("chemical reactions").head(1).iloc[0]["features"] #3 features PL2, BM25, PageRank

In [ ]:
import lightgbm as lgb
# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[1, 3, 5, 10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=10)
lmart_l_pipe = stage12 >> pt.ltr.apply_learned_model(lmart_l, form="ltr")
lmart_l_pipe.fit(tr_topics, tr_qrels, va_topics, va_qrels)
#pl2 = pt.BatchRetrieve(index, wmodel="PL2")
pt.Experiment(
    [firstpassUB, lmart_l_pipe],
    test_topics,
    test_qrels,
    ["map"],
    baseline=0,
    round={"map" : 4},
    names=["PL2 Baseline", "LambdaMART (LightGBM)" ]
)

In [ ]:
#p@5 metric
pt.Experiment(
    [firstpassUB, lmart_l_pipe],
    test_topics,
    test_qrels,
    ["P_5"],
    baseline=0,
    round={"P_5" : 4},
    names=["PL2 p_5", "LambdaMART (LightGBM) P_5" ]
)


,name,P_5,P_5 +,P_5 -,P_5 p-value
0,PL2 p_5,0.0693,NaN,NaN,NaN
1,LambdaMART (LightGBM) P_5,0.1147,22.0,6.0,0.001787


In [ ]:
from pyterrier.measures import *
pt.Experiment(
    [ firstpassUB,lmart_l_pipe],
    test_topics,
    test_qrels,
    eval_metrics=[AP],
    baseline=0,
    #round={"P_5" : 4},
    names=["pl2 baseline","LambdaMART (LightGBM) AP" ]
)


,name,AP,AP +,AP -,AP p-value
0,pl2 baseline,0.225110,NaN,NaN,NaN
1,LambdaMART (LightGBM) AP,0.410703,45.0,15.0,0.000097




In this block you should define your URL-Slashes feature, and test it. 

In [ ]:
import numpy as np

In [ ]:
#from urllib.parse import urlparse
def url_slash_feature(row):
  #count=0
  terms = row['url']
  #terms=index.getMetaIndex.getItem("url",docid)
  terms_slash=terms.count('/')
  return np.array([terms_slash])
 
url_slash=pt.apply.doc_features(url_slash_feature)
p = firstpassUB >> url_slash 
#pl2=pt.BatchRetrieve(index, wmodel="PL2")

In [ ]:
p.search("cryption")

,qid,docid,docno,url,body,rank,score,query,features
0,1,434993,G15-50-1054100,http://cs-www.ncsl.nist.gov/publications/nistp...,"References[BOCK 88] Peter Bocker, ISDN The Int...",0,8.382679,cryption,[6]
1,1,494954,G17-68-2584616,http://www.ncs.gov/n2/content/technote/tnv7n4/...,OFFICE OF THE MANAGER ...,1,5.182290,cryption,[7]
2,1,515873,G18-46-1865362,http://socialsecurity.gov/employer/Repfal00.pdf,SSA / I R SSocial SecurityAdministrationIntern...,2,5.029155,cryption,[4]
3,1,418765,G14-90-3191980,http://cs-www.ncsl.nist.gov/publications/nistb...,November 1997INTERNET ...,3,4.695244,cryption,[5]
4,1,580563,G20-84-2477732,http://itos.gsfc.nasa.gov/ITOS/remcmd.pdf,Remote Commanding Documentation ...,4,4.330407,cryption,[4]
5,1,88187,G02-78-3621877,http://cs-www.ncsl.nist.gov/ipsec/papers/aes-d...,Network Working Group ...,5,3.716963,cryption,[5]
6,1,457024,G16-34-3764782,http://w3.access.gpo.gov/bxa/ear/txt/734.txt,Part 734--Scope of the Export Administration R...,6,3.670543,cryption,[6]
7,1,424551,G15-11-3633588,http://cs-www.ncsl.nist.gov/publications/nistp...,Special Publication 800-41 Guidelines on Firew...,7,1.530188,cryption,[6]
8,1,427549,G15-22-3805523,http://cs-www.ncsl.nist.gov/publications/nistp...,Security Issues in the Database Language SQLW....,8,1.341454,cryption,[6]
9,1,567214,G20-36-0506919,http://cs-www.ncsl.nist.gov/staff/jansen/IEEEa...,I N T E L L I G E N T A G E N T SAgents for ...,9,1.299375,cryption,[5]


#### (i) URL-Slashes as a PL2 re-ranker


In [ ]:
#reranking the pl2 with the url slash feature
#rerank_pl2=url_slash_feature >> pt.ltr.feature_to_score(0)
pl2_rerank = p >> pt.ltr.feature_to_score(0)

In [ ]:
#map metric using test data
pt.Experiment(
    [firstpassUB,pl2_rerank],
    test_topics,
    test_qrels,
    ["map"],
    baseline=0,
    round={"map" : 4},
    names=["pl2","reranked pl2" ]
)



,name,map,map +,map -,map p-value
0,pl2,0.2251,NaN,NaN,NaN
1,reranked pl2,0.0022,0.0,70.0,3.386873e-08


In [ ]:
#p_5 metric using test data
pt.Experiment(
    [firstpassUB,pl2_rerank],
    test_topics,
    test_qrels,
    ["P_5"],
    baseline=0,
    round={"P_5" : 4},
    names=["pl2","reranked pl2" ]
)


,name,P_5,P_5 +,P_5 -,P_5 p-value
0,pl2,0.0693,NaN,NaN,NaN
1,reranked pl2,0.0000,0.0,26.0,2.216676e-08


#### (ii) URL-Slashes within an LTR model



In [ ]:
#YOUR SOLUTION
import lightgbm as lgb
# this configures LightGBM as LambdaMART
lmart_l_url_slash= lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[1, 3, 5, 10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=10)



In [ ]:
#add the feature to the LTR pipeline 
lmart_l_pipe_url_slash = firstpassUB >> stdfeatures ** url_slash >> pt.ltr.apply_learned_model(lmart_l_url_slash, form="ltr") #p is the pipeline witht the add url_slash_feature
lmart_l_pipe_url_slash.fit(tr_topics, tr_qrels, va_topics, va_qrels)
#pl2 = pt.BatchRetrieve(index, wmodel="PL2")
pt.Experiment(
    [lmart_l_pipe,lmart_l_pipe_url_slash],
    test_topics,
    test_qrels,
    ["map"],
    baseline=0,
    round={"map" : 4},
    names=["LambdaMART (LightGBM)" ,"LambdaMART url slash Pipeline"]
)


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's ndcg@1: 0.277778
[2]	valid_0's ndcg@1: 0.296296
[3]	valid_0's ndcg@1: 0.333333
[4]	valid_0's ndcg@1: 0.351852
[5]	valid_0's ndcg@1: 0.388889
[6]	valid_0's ndcg@1: 0.407407
[7]	valid_0's ndcg@1: 0.407407
[8]	valid_0's ndcg@1: 0.592593
[9]	valid_0's ndcg@1: 0.666667
[10]	valid_0's ndcg@1: 0.685185


,name,map,map +,map -,map p-value
0,LambdaMART (LightGBM),0.4107,NaN,NaN,NaN
1,LambdaMART url slash Pipeline,0.4577,23.0,27.0,0.312466


In [ ]:
#p_5 metric with test data with lmart and added feature
pt.Experiment(
    [lmart_l_pipe,lmart_l_pipe_url_slash],
    test_topics,
    test_qrels,
    ["P_5"],
    baseline=0,
    round={"P_5" : 4},
    names=["LambdaMART (LightGBM)" ,"LambdaMART url slash Pipeline"]
)

,name,P_5,P_5 +,P_5 -,P_5 p-value
0,LambdaMART (LightGBM),0.1147,NaN,NaN,NaN
1,LambdaMART url slash Pipeline,0.1253,9.0,6.0,0.349206


In [ ]:
#p_5 metric with test data with lmart and added feature
pt.Experiment(
    [lmart_l_pipe,lmart_l_pipe_url_slash],
    test_topics,
    test_qrels,
    [AP],
    baseline=0,
    #round={"P_5" : 4},
    names=["LambdaMART (LightGBM)" ,"LambdaMART url slash Pipeline"]
)

,name,AP,AP +,AP -,AP p-value
0,LambdaMART (LightGBM),0.410703,NaN,NaN,NaN
1,LambdaMART url slash Pipeline,0.457690,23.0,27.0,0.312466


In [ ]:
lmart_l_url_slash.feature_importances_

array([166124.37046814, 233406.06686401, 289893.13668823,  42378.61920166])

 (b) URL Type Feature


In [ ]:
from urllib.parse import urlparse
ihatethis=urlparse('http://www.cwi.nl/%7Eguido/Python.html')
ihatethis.path.split("/")

['', '%7Eguido', 'Python.html']

In [ ]:
from urllib.parse import urlparse
def url_type_feature(row):
  url=row['url']
  parsed_url=urlparse(url)
  # 1 - domain name optionally index.html
  if (parsed_url.path=="" or parsed_url.path=="/" or parsed_url.path=="/index.html"):
    url_type = 1
  # 2 - domain name & directory, or optionally index.html
  elif (len(parsed_url.path.split("/"))==3 and (parsed_url.path.split("/")[2]=='' or parsed_url.path.split("/")[2]=="index.html")):
    url_type = 2
  # 3 - deep path but not ending other than index.html
  elif (len(parsed_url.path.split("/"))>3 and (parsed_url.path.split("/")[-1]=="index.html" or "." not in parsed_url.path.split("/")[-1])):
    url_type = 3
  # 4 - anything other than ending index.html
  elif (len(parsed_url.path.split("/"))>=2 and parsed_url.path.split("/")[-1]!='index.html'):
    url_type=4
  return (np.array([url_type]))


In [ ]:
#integrate the feature to the firstpassUB model
url_type= pt.apply.doc_features(url_type_feature)
url_type_pipeline=firstpassUB>>url_type


In [ ]:
url_type_pipeline.search("cryption")

,qid,docid,docno,url,body,rank,score,query,features
0,1,434993,G15-50-1054100,http://cs-www.ncsl.nist.gov/publications/nistp...,"References[BOCK 88] Peter Bocker, ISDN The Int...",0,8.382679,cryption,[4]
1,1,494954,G17-68-2584616,http://www.ncs.gov/n2/content/technote/tnv7n4/...,OFFICE OF THE MANAGER ...,1,5.182290,cryption,[4]
2,1,515873,G18-46-1865362,http://socialsecurity.gov/employer/Repfal00.pdf,SSA / I R SSocial SecurityAdministrationIntern...,2,5.029155,cryption,[4]
3,1,418765,G14-90-3191980,http://cs-www.ncsl.nist.gov/publications/nistb...,November 1997INTERNET ...,3,4.695244,cryption,[4]
4,1,580563,G20-84-2477732,http://itos.gsfc.nasa.gov/ITOS/remcmd.pdf,Remote Commanding Documentation ...,4,4.330407,cryption,[4]
5,1,88187,G02-78-3621877,http://cs-www.ncsl.nist.gov/ipsec/papers/aes-d...,Network Working Group ...,5,3.716963,cryption,[4]
6,1,457024,G16-34-3764782,http://w3.access.gpo.gov/bxa/ear/txt/734.txt,Part 734--Scope of the Export Administration R...,6,3.670543,cryption,[4]
7,1,424551,G15-11-3633588,http://cs-www.ncsl.nist.gov/publications/nistp...,Special Publication 800-41 Guidelines on Firew...,7,1.530188,cryption,[4]
8,1,427549,G15-22-3805523,http://cs-www.ncsl.nist.gov/publications/nistp...,Security Issues in the Database Language SQLW....,8,1.341454,cryption,[4]
9,1,567214,G20-36-0506919,http://cs-www.ncsl.nist.gov/staff/jansen/IEEEa...,I N T E L L I G E N T A G E N T SAgents for ...,9,1.299375,cryption,[4]


In [ ]:
#checking
url_type_pipeline.search("aaie")

,qid,docid,docno,url,body,rank,score,query,features
0,1,543541,G19-52-0995113,http://sunshine.jpl.nasa.gov/AAIE%20Site%20%c4...,AAIE Photo Album The Jet Propulsion Labo...,0,16.253916,aaie,[3]
1,1,88532,G02-80-0379929,http://sunshine.jpl.nasa.gov/1rst%20Tier/Photo...,C ol o Photo Album This section is f...,1,13.002429,aaie,[4]
2,1,301428,G10-61-1895354,http://www.cdpr.ca.gov/docs/ipminov/01awards.htm,The 2001 IPM Innovators Awards The 2001 A...,2,6.478513,aaie,[4]
3,1,375914,G13-35-3399834,http://www.cdpr.ca.gov/docs/pressrls/9pestinno...,Media Contacts: Glenn Brank 916/445-3974 ...,3,5.839415,aaie,[4]
4,1,51341,G01-54-3873617,http://goldmine.cde.ca.gov/calendar/,BODY { margin-left : 0; margin-...,4,3.442986,aaie,[2]
5,1,430800,G15-34-2524281,http://www.cde.ca.gov/calendar/,BODY { margin-left : 0; margin-...,5,3.429759,aaie,[2]


#### (i) URL Type as a PL2 re-ranker



In [ ]:
#YOUR SOLUTION
pl2_rerank_urltype = url_type_pipeline >> pt.ltr.feature_to_score(0)
#pl2=pt.BatchRetrieve(index, wmodel="PL2")
#p2.search("chemical reactions")

In [ ]:
pt.Experiment(
    [firstpassUB,pl2_rerank_urltype],
    test_topics,
    test_qrels,
    ["map"],
    baseline=0,
    round={"map" : 4},
    names=["pl2","pl2 with urlType"]
)

,name,map,map +,map -,map p-value
0,pl2,0.2251,NaN,NaN,NaN
1,pl2 with urlType,0.0013,1.0,69.0,2.986062e-08


In [ ]:
pt.Experiment(
    [firstpassUB,pl2_rerank_urltype],
    test_topics,
    test_qrels,
    ["P_5"],
    baseline=0,
    round={"P_5" : 4},
    names=["pl2","pl2 with urlType"]
)

,name,P_5,P_5 +,P_5 -,P_5 p-value
0,pl2,0.0693,NaN,NaN,NaN
1,pl2 with urlType,0.0000,0.0,26.0,2.216676e-08


#### (ii) URL Type within an LTR model


In [ ]:
p2= firstpassUB >> stdfeatures ** url_type

In [ ]:
#testing the feature
p2.search("chemical reactions")

,qid,docid,docno,url,body,rank,score,query,features
0,1,513586,G18-38-1767991,http://www.boulder.nist.gov/div838/tar/file03....,NIST - Physical and Chemical Properties Divi...,0,12.755546,chemical reactions,"[12.755545561073266, 3.0924078763629836, 0.000..."
1,1,38544,G01-14-2537005,http://www.labtrain.noaa.gov/shemtfa/chemhaz/n...,. ...,1,11.906524,chemical reactions,"[11.90652405775751, 10.789390732195702, 0.0002..."
2,1,707122,G26-06-3754605,http://www.aps.anl.gov/xfd/tech/safetyenvelope...,APS Experiment Safety Envelope 6: Chemicals ...,2,11.877550,chemical reactions,"[11.877549705152717, 0.0, 2.3513691708978857e-..."
3,1,382754,G13-59-3981168,http://response.restoration.noaa.gov/chemaids/...,"""); } else { document.write(...",3,11.858475,chemical reactions,"[11.858474825809738, 0.0, 5.346185439867095e-0..."
4,1,70292,G02-16-2617043,http://www.symp14.nist.gov/PDF/COR04MAY.PDF,A Database of Chemical Reactions Designed to A...,4,11.731490,chemical reactions,"[11.731489696645252, 0.0, 3.778848211693258e-0..."
...,...,...,...,...,...,...,...,...,...
995,1,246965,G08-68-4141101,http://en-env.llnl.gov/asd/pinatub.html,The Chemical and Radiative Effects of the Moun...,995,6.290707,chemical reactions,"[6.290706693490568, 3.3517926096567896, 1.3237..."
996,1,611136,G22-04-3955177,http://eospso.gsfc.nasa.gov/ftp_docs/Ch7.pdf,Chapter 7 ...,996,6.289822,chemical reactions,"[6.28982231530779, 0.0, 5.268805694263197e-06,..."
997,1,594957,G21-38-0191596,http://www.oit.doe.gov/news/oittimes/wn02/wn02...,search ...,997,6.287830,chemical reactions,"[6.287829625241223, 0.0, 4.7705552713888965e-0..."
998,1,280944,G09-85-3411646,http://www.ig.doe.gov/pdf/chemfina.pdf,INS-O-00-01 I N S P E C T I O N ...,998,6.287759,chemical reactions,"[6.287758720877289, 0.0, 3.6757891813812704e-0..."


In [ ]:
#doing the same thing but with url_type pipeline
import lightgbm as lgb
# this configures LightGBM as LambdaMART
lmart_l_url_type = lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[1, 3, 5, 10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=10)
lmart_l_pipe_url_type = firstpassUB >> stdfeatures ** url_type >> pt.ltr.apply_learned_model(lmart_l_url_type, form="ltr")
lmart_l_pipe_url_type.fit(tr_topics, tr_qrels, va_topics, va_qrels)
#pl2 = pt.BatchRetrieve(index, wmodel="PL2")
pt.Experiment(
    [lmart_l_pipe,lmart_l_pipe_url_type],
    test_topics,
    test_qrels,
    ["map"],
    baseline=0,
    round={"map" : 4},
    names=[ "LambdaMART (LightGBM)" ,"LambdaMART (LightGBM) with urlType" ]
)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's ndcg@1: 0.296296
[2]	valid_0's ndcg@1: 0.351852
[3]	valid_0's ndcg@1: 0.388889
[4]	valid_0's ndcg@1: 0.388889
[5]	valid_0's ndcg@1: 0.407407
[6]	valid_0's ndcg@1: 0.407407
[7]	valid_0's ndcg@1: 0.407407
[8]	valid_0's ndcg@1: 0.407407
[9]	valid_0's ndcg@1: 0.555556
[10]	valid_0's ndcg@1: 0.574074


,name,map,map +,map -,map p-value
0,LambdaMART (LightGBM),0.4107,NaN,NaN,NaN
1,LambdaMART (LightGBM) with urlType,0.4897,29.0,17.0,0.027438


In [ ]:
#p_5 metric
pt.Experiment(
    [lmart_l_pipe,lmart_l_pipe_url_type],
    test_topics,
    test_qrels,
    ["P_5"],
    baseline=0,
    round={"P_5" : 4},
    names=["LambdaMART (LightGBM)","LambdaMART (LightGBM) with urlType" ]
)

,name,P_5,P_5 +,P_5 -,P_5 p-value
0,LambdaMART (LightGBM),0.1147,NaN,NaN,NaN
1,LambdaMART (LightGBM) with urlType,0.1413,11.0,1.0,0.003257


In [ ]:
pt.Experiment(
    [lmart_l_pipe,lmart_l_pipe_url_type],
    test_topics,
    test_qrels,
    [AP],
    baseline=0,
    #round={"P_5" : 4},
    names=["LambdaMART (LightGBM)","LambdaMART (LightGBM) with urlType" ]
)

,name,AP,AP +,AP -,AP p-value
0,LambdaMART (LightGBM),0.410703,NaN,NaN,NaN
1,LambdaMART (LightGBM) with urlType,0.489735,29.0,17.0,0.027438


In [ ]:
lmart_l_url_type.feature_importances_

array([140248.29094696, 163487.00793457, 380687.99206543,  40292.72741699])

# Q3 Proximity Search Feature



In [ ]:
from statistics import mean

In [ ]:

from string import punctuation


def remove_punc(sentence):
  terms = sentence.split(" ")     
  terms = [t for t in terms if not t in list(punctuation)]
  return " ".join(terms)

def avgmindist(query : str,document : str): #-> float
  #update your implementation here.
  query = query.lower()
  document = document.lower()
  query = remove_punc(query)
  document = remove_punc(document)
  t = query.split()
  dist_tokens =[]
  if(len(t)>1):
    for i in range(0,len(t)-1):
      dist_tokens.append(MinDist(t[i],t[i+1],document))    
    
    while -1 in dist_tokens: dist_tokens.remove(-1)
    if len(dist_tokens)>0:
      avgdistance = float(mean(dist_tokens))
    else:
      avgdistance = float(len(document.split()))
  else:
    avgdistance = float(len(document.split()))
  return np.array([avgdistance])

def MinDist(a,b,d):
  d = d.split()
  distance=[]
  a_index = [i for i, value in enumerate(d) if value == a]
  b_index = [i for i, value in enumerate(d) if value == b]
  for index_a in a_index:
    for index_b in b_index:
      distance.append(abs(index_b-index_a-1))
  if len(distance)>0:
    distance = min(distance)
  else:
    distance = -1
  return distance


In [ ]:
#DO NOT ALTER THIS CELL
TEST_CASES = [
  ('fermilab directory', 45, 567257), #1
  ('webcam', 45, 567257), #2
  ('DOM surface', 384034, 388292), #3
  ('DOM surface', 45, 384034), #4
  ('DOM surface document', 388292, 384034), #5
  ('DOM software AMANDA', 639302, 384034), #6
  ('fermilab directory', 388292, 384034), #7
  ('trigger data', 596532, 639302), #8
  ('underlying hardware', 384034, 333649) #9
]

def run_test_cases():
  docno=0
  body=3
  for i, (query, docid1, docid2) in enumerate(TEST_CASES):
    meta1 = index.getMetaIndex().getAllItems(docid1)
    meta2 = index.getMetaIndex().getAllItems(docid2)
    s1 = avgmindist(query, meta1[body])
    s2 = avgmindist(query, meta2[body])
    if s1 > s2:
      result = meta1[docno]
      cmpD = "%s > %s" % (meta1[docno],meta2[docno])
    elif s2 > s1:
      result = meta2[docno]
      cmpD = "%s > %s" % (meta2[docno],meta1[docno])
    else:
      result = "EQUAL"
      cmpD = "%s == %s" % (meta1[docno],meta2[docno])
    print("TEST CASE %d result %s " % (i+1, result))

run_test_cases()

TEST CASE 1 result G20-36-1335992 
TEST CASE 2 result G20-36-1335992 
TEST CASE 3 result G13-80-1271020 
TEST CASE 4 result G00-00-0478398 
TEST CASE 5 result G13-80-1271020 
TEST CASE 6 result G23-17-3674898 
TEST CASE 7 result G13-80-1271020 
TEST CASE 8 result G21-44-1000362 
TEST CASE 9 result G13-64-2457111 


 integrate avgmindist() function into a new LTR model, and compare its MAP & P@5 performance to the LTR baseline.

In [ ]:
#integrating the avgmindist for row['query'] and row['body']

min_dist= pt.apply.doc_features(lambda row: avgmindist(row['query'],row['body']))

#combining min_dist to original firstpassUB model

min_dist_pipe=firstpassUB>>min_dist

In [ ]:
min_dist_pipe.search("chemical reactions")

,qid,docid,docno,url,body,rank,score,query,features
0,1,513586,G18-38-1767991,http://www.boulder.nist.gov/div838/tar/file03....,NIST - Physical and Chemical Properties Divi...,0,12.755546,chemical reactions,[0.0]
1,1,38544,G01-14-2537005,http://www.labtrain.noaa.gov/shemtfa/chemhaz/n...,. ...,1,11.906524,chemical reactions,[2.0]
2,1,707122,G26-06-3754605,http://www.aps.anl.gov/xfd/tech/safetyenvelope...,APS Experiment Safety Envelope 6: Chemicals ...,2,11.877550,chemical reactions,[0.0]
3,1,382754,G13-59-3981168,http://response.restoration.noaa.gov/chemaids/...,"""); } else { document.write(...",3,11.858475,chemical reactions,[584.0]
4,1,70292,G02-16-2617043,http://www.symp14.nist.gov/PDF/COR04MAY.PDF,A Database of Chemical Reactions Designed to A...,4,11.731490,chemical reactions,[0.0]
...,...,...,...,...,...,...,...,...,...
995,1,246965,G08-68-4141101,http://en-env.llnl.gov/asd/pinatub.html,The Chemical and Radiative Effects of the Moun...,995,6.290707,chemical reactions,[350.0]
996,1,611136,G22-04-3955177,http://eospso.gsfc.nasa.gov/ftp_docs/Ch7.pdf,Chapter 7 ...,996,6.289822,chemical reactions,[246.0]
997,1,594957,G21-38-0191596,http://www.oit.doe.gov/news/oittimes/wn02/wn02...,search ...,997,6.287830,chemical reactions,[430.0]
998,1,280944,G09-85-3411646,http://www.ig.doe.gov/pdf/chemfina.pdf,INS-O-00-01 I N S P E C T I O N ...,998,6.287759,chemical reactions,[396.0]


In [ ]:
lmart_l_mindist = lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[1, 3, 5, 10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=10)

lmart_l_pipe_mindist = firstpassUB >> stdfeatures ** min_dist >> pt.ltr.apply_learned_model(lmart_l_mindist, form="ltr")
lmart_l_pipe_mindist.fit(tr_topics, tr_qrels, va_topics, va_qrels)

pt.Experiment(
        [lmart_l_pipe,lmart_l_pipe_mindist],
        test_topics,
        test_qrels,
        ["map"],
        round={"map" : 4},
        baseline = 0,
        names=["LambdaMART (LightGBM)", "LambdaMART (LightGBM) with mindist"]
        )


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's ndcg@1: 0.277778
[2]	valid_0's ndcg@1: 0.351852
[3]	valid_0's ndcg@1: 0.351852
[4]	valid_0's ndcg@1: 0.351852
[5]	valid_0's ndcg@1: 0.37037
[6]	valid_0's ndcg@1: 0.37037
[7]	valid_0's ndcg@1: 0.37037
[8]	valid_0's ndcg@1: 0.37037
[9]	valid_0's ndcg@1: 0.37037
[10]	valid_0's ndcg@1: 0.37037


,name,map,map +,map -,map p-value
0,LambdaMART (LightGBM),0.4107,NaN,NaN,NaN
1,LambdaMART (LightGBM) with mindist,0.4296,21.0,16.0,0.400621


In [ ]:
pt.Experiment(
        [lmart_l_pipe,lmart_l_pipe_mindist],
        test_topics,
        test_qrels,
        ["P_5"],
        baseline= 0,
        round={"P_5" : 4},
        names=["LambdaMART (LightGBM)", "LambdaMART (LightGBM) with mindist" ]
        )


,name,P_5,P_5 +,P_5 -,P_5 p-value
0,LambdaMART (LightGBM),0.1147,NaN,NaN,NaN
1,LambdaMART (LightGBM) with mindist,0.1173,3.0,2.0,0.657751


 A 5-feature Learning-to-Rank Model

LightGBM LambdaMART technique where you include both your added features (URL Type and AvgMinDist) along the 3 initial features inc PL2 sample (5 features in total). 


In [ ]:
import lightgbm as lgb
# this configures LightGBM as LambdaMART
lmart_l_final = lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[1, 3, 5, 10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=10)
lmart_l_pipe_final = firstpassUB >> stdfeatures ** url_type ** min_dist >> pt.ltr.apply_learned_model(lmart_l_final, form="ltr") #urltype + mindist
lmart_l_pipe_final.fit(tr_topics, tr_qrels, va_topics, va_qrels)
#pl2 = pt.BatchRetrieve(index, wmodel="PL2")
pt.Experiment(
    [lmart_l_pipe,lmart_l_pipe_final],
    test_topics,
    test_qrels,
    ["map"],
    baseline=0,
    round={"map" : 4},
    names=[ "LambdaMART (LightGBM)" ,"LambdaMART (LightGBM) with urlType and mindist" ]
)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's ndcg@1: 0.314815
[2]	valid_0's ndcg@1: 0.37037
[3]	valid_0's ndcg@1: 0.407407
[4]	valid_0's ndcg@1: 0.425926
[5]	valid_0's ndcg@1: 0.425926
[6]	valid_0's ndcg@1: 0.425926
[7]	valid_0's ndcg@1: 0.425926
[8]	valid_0's ndcg@1: 0.611111
[9]	valid_0's ndcg@1: 0.648148
[10]	valid_0's ndcg@1: 0.611111


,name,map,map +,map -,map p-value
0,LambdaMART (LightGBM),0.4107,NaN,NaN,NaN
1,LambdaMART (LightGBM) with urlType and mindist,0.5164,35.0,18.0,0.022042


In [ ]:
pt.Experiment(
    [lmart_l_pipe,lmart_l_pipe_final],
    test_topics,
    test_qrels,
    ["P_5"],
    baseline=0,
    round={"P_5" : 4},
    names=[ "LambdaMART (LightGBM)" ,"LambdaMART (LightGBM) with urlType and mindist" ]
)

,name,P_5,P_5 +,P_5 -,P_5 p-value
0,LambdaMART (LightGBM),0.1147,NaN,NaN,NaN
1,LambdaMART (LightGBM) with urlType and mindist,0.1307,10.0,5.0,0.158688


In [ ]:
#feature importance of the final pipeline

lmart_l_final.feature_importances_

array([154357.310112  ,  99065.57434082, 424109.64147949,  33787.29968262,
         9895.2989502 ])